<a href="https://colab.research.google.com/github/VineetSivadasan/houses/blob/main/Multivariate_Logistic_Regression_Cirrhosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap
dfCirrhosis = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Cirrhosis.csv')
print(dfCirrhosis)

In [ ]:
#checking to see if there are NaN values in any cell
checking_NaN = dfCirrhosis.isnull().values.any() 
checking_NaN

In [ ]:
#print the rows where NaN values appear in the cells
print_NaN_rows = dfCirrhosis[dfCirrhosis.isnull().any(axis=1)]  
print_NaN_rows.shape

In [ ]:
dfCirrhosis.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [ ]:
dfCirrhosis.shape

In [ ]:
corr_matrix = dfCirrhosis.corr()
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
del dfCirrhosis["bilirubin"]

In [ ]:
del dfCirrhosis["status"]

In [ ]:
del dfCirrhosis["index"]

In [ ]:
del dfCirrhosis["edema"]

In [ ]:
del dfCirrhosis["ascites"]

In [ ]:
dfCirrhosis

In [ ]:
dfCirrhosis["duration"] = (dfCirrhosis["duration"]-dfCirrhosis["duration"].min())/(dfCirrhosis["duration"].max()-dfCirrhosis["duration"].min())
dfCirrhosis["drug"] = (dfCirrhosis["drug"]-dfCirrhosis["drug"].min())/(dfCirrhosis["drug"].max()-dfCirrhosis["drug"].min())
dfCirrhosis["age"] = (dfCirrhosis["age"]-dfCirrhosis["age"].min())/(dfCirrhosis["age"].max()-dfCirrhosis["age"].min())
dfCirrhosis["sex"] = (dfCirrhosis["sex"]-dfCirrhosis["sex"].min())/(dfCirrhosis["sex"].max()-dfCirrhosis["sex"].min())
#dfCirrhosis["ascites"] = (dfCirrhosis["ascites"]-dfCirrhosis["ascites"].min())/(dfCirrhosis["ascites"].max()-dfCirrhosis["ascites"].min())
dfCirrhosis["hepatomology"] = (dfCirrhosis["hepatomology"]-dfCirrhosis["hepatomology"].min())/(dfCirrhosis["hepatomology"].max()-dfCirrhosis["hepatomology"].min())
dfCirrhosis["spiders"] = (dfCirrhosis["spiders"]-dfCirrhosis["spiders"].min())/(dfCirrhosis["spiders"].max()-dfCirrhosis["spiders"].min())
#dfCirrhosis["edema"] = (dfCirrhosis["edema"]-dfCirrhosis["edema"].min())/(dfCirrhosis["edema"].max()-dfCirrhosis["edema"].min())
dfCirrhosis["cholesterol"] = (dfCirrhosis["cholesterol"]-dfCirrhosis["cholesterol"].min())/(dfCirrhosis["cholesterol"].max()-dfCirrhosis["cholesterol"].min())
dfCirrhosis["albumin"] = (dfCirrhosis["albumin"]-dfCirrhosis["albumin"].min())/(dfCirrhosis["albumin"].max()-dfCirrhosis["albumin"].min())
dfCirrhosis["copper"] = (dfCirrhosis["copper"]-dfCirrhosis["copper"].min())/(dfCirrhosis["copper"].max()-dfCirrhosis["copper"].min())
dfCirrhosis["phosphatase"] = (dfCirrhosis["phosphatase"]-dfCirrhosis["phosphatase"].min())/(dfCirrhosis["phosphatase"].max()-dfCirrhosis["phosphatase"].min())
dfCirrhosis["SGOT"] = (dfCirrhosis["SGOT"]-dfCirrhosis["SGOT"].min())/(dfCirrhosis["SGOT"].max()-dfCirrhosis["SGOT"].min())
dfCirrhosis["triglicerides"] = (dfCirrhosis["triglicerides"]-dfCirrhosis["triglicerides"].min())/(dfCirrhosis["triglicerides"].max()-dfCirrhosis["triglicerides"].min())
dfCirrhosis["platelets"] = (dfCirrhosis["platelets"]-dfCirrhosis["platelets"].min())/(dfCirrhosis["platelets"].max()-dfCirrhosis["platelets"].min())
dfCirrhosis["prothrombin"] = (dfCirrhosis["prothrombin"]-dfCirrhosis["prothrombin"].min())/(dfCirrhosis["prothrombin"].max()-dfCirrhosis["prothrombin"].min())

In [ ]:
dfCirrhosis

In [ ]:
corr_matrix = dfCirrhosis.corr()
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
# constructing training, validation and test sets
def data_train_validate_test(df, training_set_percent=0.8, validation_set_percent=0.1, seed=None):
  np.random.seed(seed)  #set a random seed number such that the same index sampled here is repeated 
  permutation = np.random.permutation(df.index)  #random selection of the dataframe index
  print(permutation)
  length = len(df.index)  #size of index of dataset
  #print(length)
  training_set_end = int(training_set_percent * length)  #size of the training set
  print(training_set_end)
  validation_set_end = int(validation_set_percent * length) + training_set_end  #size of the validation set + training set
  print(validation_set_end)
  training = df.loc[permutation[:training_set_end]] #select random rows up to the size of the training set to form the training dataset
  validation = df.loc[permutation[training_set_end:validation_set_end]] #select random rows up to the size of validation set to form the validation dataset
  test = df.loc[permutation[validation_set_end:]] #The remaining rows from the validation set onwards to the end will be the test dataset
  return training, validation, test

In [ ]:
training, validation, test = data_train_validate_test(dfCirrhosis)


In [ ]:
cmap_bold = ListedColormap(["#FF0000","#0000FF"])
cmap_light = ListedColormap(["#FFBBBB","#BBBBFF"])

In [ ]:
training.shape, validation.shape, test.shape

In [ ]:
def softmax(h):
  return (np.exp(h.T)/np.sum(np.exp(h), axis=1)).T

def cross_entropy(Y,P_hat):
  return -(1/len(Y))*np.sum(np.sum(Y*np.log(P_hat),axis=1),axis=0)

def accuracy(y,y_hat):
  return np.mean(y==y_hat)

def indices_to_one_hot(data, nb_classes):
  targets = np.array(data).reshape(-1)
  targets = targets.astype(int)
  return np.eye(nb_classes)[targets]

In [ ]:
class MVLogisticRegression():

  def __init__(self, thresh =0.5):
    self.thresh = thresh

  def fit(self, X, y, eta=2e-1, epochs = 1e3, show_curve = False):
    epochs = int(epochs)
    N,D =X.shape
    K = len(np.unique(y))
    y_values = np.unique(y, return_index=False)
    Y = indices_to_one_hot(y-1, K).astype(int)
    self.W = np.random.randn(D,K)
    self.B = np.random.randn(1,K)
    J=np.zeros(int(epochs))

    for epoch in range(epochs):
      P_hat = self.__forward__(X)
      J[epoch] = cross_entropy(Y,P_hat)
      self.W -= eta*(1/N)*X.T@(P_hat-Y)
      self.B -= eta*(1/N)*np.sum(P_hat-Y,axis=0)

    if show_curve:
      plt.figure()
      plt.plot(J)
      plt.xlabel('epochs')
      plt.ylabel("$\matchcal{J}")
      plt.title("Training Curve")
      plt.show()
  
  def __forward__(self, X):
    return softmax(X@self.W + self.B)
  
  def predict(self,X):
    return np.argmax(self.__forward__(X), axis=1)


In [ ]:
training

In [ ]:
y_training = training['stage'].to_numpy()
X_training = training[['duration', 'drug',	'age',	'sex',	'hepatomology',	'spiders',	'cholesterol',	'albumin',	'copper',	'phosphatase',	'SGOT',	'triglicerides',	'platelets',	'prothrombin']].to_numpy()

In [ ]:
X_training

In [ ]:
y_training

In [ ]:
logreg = MVLogisticRegression()

In [ ]:
logreg.fit(X_training, y_training, eta=7e-3, epochs =3e3, show_curve=True)
y_hat = logreg.predict(X_training)
print(f"Training Accuracy: {accuracy(y_training, y_hat):0.4f}")

In [ ]:
    plt.figure()
    plt.scatter(X_training[:,4], X_training[:,7], c = y_training, alpha = 200)
    plt.xlabel("hepatomology and albumin")
    plt.ylabel("stage probability")
    plt.show()

In [ ]:
y_validation = validation['stage'].to_numpy()
X_validation = validation[['duration', 'drug',	'age',	'sex',	'hepatomology',	'spiders',	'cholesterol',	'albumin',	'copper',	'phosphatase',	'SGOT',	'triglicerides',	'platelets',	'prothrombin']].to_numpy()

In [ ]:
y_hat_validation = logreg.predict(X_validation)
print(f"validation Accuracy: {accuracy(y_validation, y_hat_validation):0.4f}")

In [ ]:
    plt.figure()
    plt.scatter(X_validation[:,4], X_validation[:,7], c = y_validation, alpha = 200)
    plt.xlabel("hepatomology and albumin")
    plt.ylabel("stage probability")
    plt.show()

In [ ]:
y_test = test['stage'].to_numpy()
X_test = test[['duration', 'drug',	'age',	'sex',	'hepatomology',	'spiders',	'cholesterol',	'albumin',	'copper',	'phosphatase',	'SGOT',	'triglicerides',	'platelets',	'prothrombin']].to_numpy()

In [ ]:
y_hat_test = logreg.predict(X_test)
print(f"Test Accuracy: {accuracy(y_test, y_hat_test):0.4f}")

In [ ]:
    plt.figure()
    plt.scatter(X_test[:,4], X_test[:,7], c = y_test, alpha = 200)
    plt.xlabel("hepatomology and albumin")
    plt.ylabel("stage probability")
    plt.show()